In [1]:
import pandas as pd
import os,sys, imp, glob
sys.path.append('./src/')
import doc_utils_clean as doc_utils
import feature_utils, common_utils
import defines
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn_crfsuite.utils import flatten

In [2]:
dir_name="duplicate"

## Merge sent pos count & sent db with all features

In [5]:
sent_db= common_utils.concat_dbs(dir_name,"sent_db")

In [18]:
sent_db.head()

,file_idx,sent_idx,is_question,text,sent_idx_in_block,block_idx,is_nar,doc_idx,par_db_idx,par_idx_in_doc,par_pos_in_doc,par_type,block_type,nar_idx,sent_len,is_client,sent_idx_in_par,sent_pos_in_par,sent_pos_in_doc
0,1,0,0.000000,אז היום אנחנו נפגשים פה ב 123 אבל שבוע הבא בח...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.004878,therapist,not_nar,0.000000,50.000000,0,0,1.000000,0.001631
1,1,1,0.000000,כן,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.009756,client,not_nar,0.000000,2.000000,1,0,1.000000,0.003263
2,1,2,1.000000,אוקיי מה שלומך,0.000000,2.000000,0.000000,1.000000,2.000000,2.000000,0.014634,therapist,not_nar,0.000000,16.000000,0,0,1.000000,0.004894
3,1,3,0.000000,ברוך השם,0.000000,3.000000,0.000000,1.000000,3.000000,3.000000,0.019512,client,not_nar,0.000000,8.000000,1,0,1.000000,0.006525
4,1,4,0.000000,מחר יש לכם יום טיול,0.000000,4.000000,0.000000,1.000000,4.000000,4.000000,0.024390,therapist,not_nar,0.000000,19.000000,0,0,1.000000,0.008157


In [19]:
sent_pos_count_db =  common_utils.concat_dbs(dir_name,"sent_pos_count_db")
sent_pos_count_db.head()

,doc_idx,sent_idx,TOKEN,POSTAG_ADVERB,POSTAG_AT,POSTAG_BN,POSTAG_BNT,POSTAG_CC,POSTAG_CD,POSTAG_CDT,...,f_per_2,f_per_3,f_per_A,f_tense_BEINONI,f_tense_FUTURE,f_tense_IMPERATIVE,f_tense_PAST,POSTAG_NEG,POSTAG_yyCM,POSTAG_TTL
0,1,0,7,0.000000,0.000000,0.142857,0.000000,0.142857,0.285714,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,1,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2,1,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
3,1,3,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
4,1,4,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [20]:
all_featuers_db=pd.merge(sent_db,sent_pos_count_db,left_on=['doc_idx','sent_idx'], right_on=['doc_idx','sent_idx'],validate='one_to_one')
all_featuers_db.head()

,file_idx,sent_idx,is_question,text,sent_idx_in_block,block_idx,is_nar,doc_idx,par_db_idx,par_idx_in_doc,...,f_per_2,f_per_3,f_per_A,f_tense_BEINONI,f_tense_FUTURE,f_tense_IMPERATIVE,f_tense_PAST,POSTAG_NEG,POSTAG_yyCM,POSTAG_TTL
0,1,0,0.000000,אז היום אנחנו נפגשים פה ב 123 אבל שבוע הבא בח...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,1,1,0.000000,כן,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2,1,2,1.000000,אוקיי מה שלומך,0.000000,2.000000,0.000000,1.000000,2.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
3,1,3,0.000000,ברוך השם,0.000000,3.000000,0.000000,1.000000,3.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
4,1,4,0.000000,מחר יש לכם יום טיול,0.000000,4.000000,0.000000,1.000000,4.000000,4.000000,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [21]:
all_featuers_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38434 entries, 0 to 38433
Data columns (total 74 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   file_idx            38434 non-null  int64  
 1   sent_idx            38434 non-null  int64  
 2   is_question         38434 non-null  float64
 3   text                38434 non-null  object 
 4   sent_idx_in_block   38434 non-null  float64
 5   block_idx           38434 non-null  float64
 6   is_nar              38434 non-null  float64
 7   doc_idx             38434 non-null  float64
 8   par_db_idx          38434 non-null  float64
 9   par_idx_in_doc      38434 non-null  float64
 10  par_pos_in_doc      38434 non-null  float64
 11  par_type            38434 non-null  object 
 12  block_type          38434 non-null  object 
 13  nar_idx             38434 non-null  float64
 14  sent_len            38434 non-null  float64
 15  is_client           38434 non-null  int64  
 16  sent

In [22]:
to_drop=['par_db_idx','doc_idx','block_type','par_type','block_idx','sent_idx_in_block','par_db_idx','text']

In [23]:
all_featuers_db.drop(to_drop,axis=1,inplace=True)

In [34]:
col_list=['file_idx','sent_idx','is_nar','TOKEN','is_client','is_question','nar_idx','sent_idx_in_par','par_idx_in_doc','sent_len']

In [35]:
all_featuers_db[col_list]=all_featuers_db[col_list].astype('int32')

In [31]:
all_featuers_db['is_question'].unique()

array([0, 1], dtype=int32)

In [43]:
all_featuers_db.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38434 entries, 0 to 38433
Data columns (total 67 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   file_idx             38434 non-null  int32  
 1   sent_idx             38434 non-null  int32  
 2   is_question          38434 non-null  int32  
 3   is_nar               38434 non-null  int32  
 4   par_idx_in_doc       38434 non-null  int32  
 5   par_pos_in_doc       38434 non-null  float64
 6   nar_idx              38434 non-null  int32  
 7   sent_len_characters  38434 non-null  int32  
 8   is_client            38434 non-null  int32  
 9   sent_idx_in_par      38434 non-null  int32  
 10  sent_pos_in_par      38434 non-null  float64
 11  sent_pos_in_doc      38434 non-null  float64
 12  sent_len_words       38434 non-null  int32  
 13  POSTAG_ADVERB        38434 non-null  float64
 14  POSTAG_AT            38434 non-null  float64
 15  POSTAG_BN            38434 non-null 

In [41]:
all_featuers_db

,file_idx,sent_idx,is_question,is_nar,par_idx_in_doc,par_pos_in_doc,nar_idx,sent_len,is_client,sent_idx_in_par,...,f_per_2,f_per_3,f_per_A,f_tense_BEINONI,f_tense_FUTURE,f_tense_IMPERATIVE,f_tense_PAST,POSTAG_NEG,POSTAG_yyCM,POSTAG_TTL
0,1,0,0,0,0,0.004878,0,50,0,0,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
1,1,1,0,0,1,0.009756,0,2,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
2,1,2,1,0,2,0.014634,0,16,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
3,1,3,0,0,3,0.019512,0,8,1,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
4,1,4,0,0,4,0.024390,0,19,0,0,...,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38429,80,525,0,0,222,0.991111,0,21,1,3,...,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
38430,80,526,0,0,222,0.991111,0,27,1,4,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
38431,80,527,0,0,222,0.991111,0,23,1,5,...,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN
38432,80,528,1,0,223,0.995556,0,19,0,0,...,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN


In [42]:
all_featuers_db.rename(columns={"sent_len": "sent_len_characters", "TOKEN": "sent_len_words"},inplace=True)

In [144]:
all_featuers_db['sent_len_words'].describe()

count   38434.000000
mean        9.307696
std         9.381784
min         1.000000
25%         4.000000
50%         6.000000
75%        12.000000
max       166.000000
Name: sent_len_words, dtype: float64

In [46]:
common_utils.save_db(all_featuers_db,dir_name,"narrative.features",True)

Saving /scratch/sim_reg9/users/zsofya/classroom/MSc/personal_study/thesis/dataframes/duplicate/narrative.features.csv, 
index True
float_format %.5f


# Find examples of PoS tags

In [95]:
sent_pos_db =  common_utils.concat_dbs(dir_name,"sent_pos_db")
sent_pos_db

,file_idx,sent_idx,FROM,sent_idx,TO,FORM,LEMMA,CPOSTAG,POSTAG,FEATS,TOKEN,f_gen,f_num,f_per,f_suf_gen,f_suf_num,f_suf_per,f_tense,doc_idx
0,1,0,0,0.000000,1,אז,אז,RB,RB,_,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
1,1,1,1,0.000000,2,היום,היום,RB,RB,_,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2,1,2,2,0.000000,3,אנחנו,הוא,PRP,PRP,gen=F|gen=M|num=P|per=1,3,M,P,1,NaN,NaN,NaN,NaN,1.000000
3,1,3,3,0.000000,4,נפגשים,נפגש,BN,BN,gen=M|num=P|per=A,4,M,P,A,NaN,NaN,NaN,NaN,1.000000
4,1,4,4,0.000000,5,פה,פה,NNP,NNP,gen=M|num=S,5,M,S,NaN,NaN,NaN,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553912,80,5827,0,529.000000,1,היה,היה,COP,COP,gen=M|num=S|per=3,1,M,S,3,NaN,NaN,NaN,NaN,NaN
553913,80,5828,1,529.000000,2,ל,ל,IN,IN,_,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553914,80,5829,2,529.000000,3,אני,אני,S_PRN,S_PRN,gen=F|gen=M|num=S|per=1,2,M,S,1,NaN,NaN,NaN,NaN,NaN
553915,80,5830,3,529.000000,4,מאוד,מאוד,RB,RB,_,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
sent_pos_db.query("POSTAG=='NNP'")['FORM'].value_counts().reset_index().loc[:3,'index'].tolist()

['XXX', 'ו', 'ש', 'ה']

In [146]:
sent_pos_count_db =  common_utils.concat_dbs(dir_name,"sent_pos_count_db")


,file_idx,sent_idx,FROM,sent_idx,TO,FORM,LEMMA,CPOSTAG,POSTAG,FEATS,TOKEN,f_gen,f_num,f_per,f_suf_gen,f_suf_num,f_suf_per,f_tense,doc_idx
0,1,0,0,0.000000,1,אז,אז,RB,RB,_,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
1,1,1,1,0.000000,2,היום,היום,RB,RB,_,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
2,1,2,2,0.000000,3,אנחנו,הוא,PRP,PRP,gen=F|gen=M|num=P|per=1,3,M,P,1,NaN,NaN,NaN,NaN,1.000000
3,1,3,3,0.000000,4,נפגשים,נפגש,BN,BN,gen=M|num=P|per=A,4,M,P,A,NaN,NaN,NaN,NaN,1.000000
4,1,4,4,0.000000,5,פה,פה,NNP,NNP,gen=M|num=S,5,M,S,NaN,NaN,NaN,NaN,NaN,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553912,80,5827,0,529.000000,1,היה,היה,COP,COP,gen=M|num=S|per=3,1,M,S,3,NaN,NaN,NaN,NaN,NaN
553913,80,5828,1,529.000000,2,ל,ל,IN,IN,_,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553914,80,5829,2,529.000000,3,אני,אני,S_PRN,S_PRN,gen=F|gen=M|num=S|per=1,2,M,S,1,NaN,NaN,NaN,NaN,NaN
553915,80,5830,3,529.000000,4,מאוד,מאוד,RB,RB,_,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [151]:
sent_pos_count_db.columns
pos_col_names = [col for col in sent_pos_count_db.columns if 'POSTAG' in col]
pos_col_names

['POSTAG_ADVERB',
 'POSTAG_AT',
 'POSTAG_BN',
 'POSTAG_BNT',
 'POSTAG_CC',
 'POSTAG_CD',
 'POSTAG_CDT',
 'POSTAG_CONJ',
 'POSTAG_COP',
 'POSTAG_DEF',
 'POSTAG_DT',
 'POSTAG_DTT',
 'POSTAG_EX',
 'POSTAG_IN',
 'POSTAG_INTJ',
 'POSTAG_JJ',
 'POSTAG_JJT',
 'POSTAG_MD',
 'POSTAG_NN',
 'POSTAG_NNP',
 'POSTAG_NNT',
 'POSTAG_P',
 'POSTAG_POS',
 'POSTAG_PREPOSITION',
 'POSTAG_PRP',
 'POSTAG_QW',
 'POSTAG_RB',
 'POSTAG_REL',
 'POSTAG_S_PRN',
 'POSTAG_TEMP',
 'POSTAG_VB',
 'POSTAG_NEG',
 'POSTAG_yyCM',
 'POSTAG_TTL']

In [153]:
f_col_names = [col for col in sent_pos_count_db.columns if 'f_' in col]
f_col_names

['f_gen_F',
 'f_gen_M',
 'f_num_D',
 'f_num_P',
 'f_num_S',
 'f_suf_gen_F',
 'f_suf_gen_M',
 'f_suf_num_P',
 'f_suf_num_S',
 'f_suf_per_1.0',
 'f_suf_per_2.0',
 'f_suf_per_3.0',
 'f_per_1',
 'f_per_2',
 'f_per_3',
 'f_per_A',
 'f_tense_BEINONI',
 'f_tense_FUTURE',
 'f_tense_IMPERATIVE',
 'f_tense_PAST']

In [154]:
pos_col_names.extend(f_col_names)
pos_col_names

['POSTAG_ADVERB',
 'POSTAG_AT',
 'POSTAG_BN',
 'POSTAG_BNT',
 'POSTAG_CC',
 'POSTAG_CD',
 'POSTAG_CDT',
 'POSTAG_CONJ',
 'POSTAG_COP',
 'POSTAG_DEF',
 'POSTAG_DT',
 'POSTAG_DTT',
 'POSTAG_EX',
 'POSTAG_IN',
 'POSTAG_INTJ',
 'POSTAG_JJ',
 'POSTAG_JJT',
 'POSTAG_MD',
 'POSTAG_NN',
 'POSTAG_NNP',
 'POSTAG_NNT',
 'POSTAG_P',
 'POSTAG_POS',
 'POSTAG_PREPOSITION',
 'POSTAG_PRP',
 'POSTAG_QW',
 'POSTAG_RB',
 'POSTAG_REL',
 'POSTAG_S_PRN',
 'POSTAG_TEMP',
 'POSTAG_VB',
 'POSTAG_NEG',
 'POSTAG_yyCM',
 'POSTAG_TTL',
 'f_gen_F',
 'f_gen_M',
 'f_num_D',
 'f_num_P',
 'f_num_S',
 'f_suf_gen_F',
 'f_suf_gen_M',
 'f_suf_num_P',
 'f_suf_num_S',
 'f_suf_per_1.0',
 'f_suf_per_2.0',
 'f_suf_per_3.0',
 'f_per_1',
 'f_per_2',
 'f_per_3',
 'f_per_A',
 'f_tense_BEINONI',
 'f_tense_FUTURE',
 'f_tense_IMPERATIVE',
 'f_tense_PAST']

In [155]:
sent_pos_count_db[pos_col_names].describe()

,POSTAG_ADVERB,POSTAG_AT,POSTAG_BN,POSTAG_BNT,POSTAG_CC,POSTAG_CD,POSTAG_CDT,POSTAG_CONJ,POSTAG_COP,POSTAG_DEF,...,f_suf_per_2.0,f_suf_per_3.0,f_per_1,f_per_2,f_per_3,f_per_A,f_tense_BEINONI,f_tense_FUTURE,f_tense_IMPERATIVE,f_tense_PAST
count,38434.000000,38434.000000,38434.000000,38434.000000,38434.000000,38434.000000,36689.000000,38434.000000,38434.000000,38434.000000,...,36410.000000,38434.000000,38434.000000,38434.000000,38434.000000,38434.000000,38434.000000,38434.000000,38434.000000,38434.000000
mean,0.004500,0.030057,0.083749,0.000864,0.051960,0.011656,0.001052,0.037623,0.023226,0.107510,...,0.001325,0.010406,0.097731,0.039345,0.169336,0.093563,0.004495,0.016659,0.002591,0.049521
std,0.049427,0.066802,0.116031,0.010521,0.094239,0.062654,0.014717,0.093793,0.064739,0.166837,...,0.020455,0.060654,0.133456,0.093112,0.175960,0.120194,0.030153,0.055963,0.031064,0.098084
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.142857,0.052632,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.142857,0.000000,0.083333,0.000000,0.000000,0.031250,0.000000,0.166667,...,0.000000,0.000000,0.166667,0.000000,0.250000,0.166667,0.000000,0.000000,0.000000,0.071429
max,1.000000,1.000000,2.000000,0.500000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,...,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000
